# CATALAN

WER METRICS ARE NOT REPRESENTATIVE HERE SINCE WE ARE TAKING JUST 20 SAMPLES AT INFERENCE.

In [2]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

model_path = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\modelos\multilanguage-catalan"
processor = Wav2Vec2Processor.from_pretrained(model_path)
model = Wav2Vec2ForCTC.from_pretrained(model_path)


d:\Users\Usuario\anaconda3\envs\new_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Usuario\AppData\Roaming\Python\Python311\site-packages\transformers\configuration_utils.py:306: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
# from this path we will do the transcription for 10 files.
path = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\clips"
import os

files = os.listdir(path)[100:120]  # Limit to the first 10 files

for file in files:
    audio_path = os.path.join(path, file)
    speech, _ = librosa.load(audio_path, sr=16000)
    #speech = np.clip(speech, -1.0, 1.0)  # Normalize the audio signal
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)


ha escrit vàries obres i articles que tracten sobre científics
i qui era que els guiava mentre acabaven de condicionar-ho tot
unides podem ha acceptat la proposta
com es representava inicialment
es mantenen els motius de protesta
podries avisar-me si surt alguna cosa nova de la situació de tram
contra quell ha d'haver un posicionament segons cruanyes
de quina escola relacionada amb la seva professió forma part
només hi ha votat en contra siu
va ser ben considerat el film
què digui al noi
aquestes corporacions eren totalment diferents entre elles
i com són actualment
què ha de ser autoritzat l'església
va tenir alguna repercussió
què diu haver-li ofert el jove
va acabar sent més important que l'original
quins artistes principals hi havia
quin càrrec tenia ell
conviuen en un mateix lloc


In [15]:
import os
import pandas as pd
import librosa
import torch

# Ruta del directorio de clips y el archivo validated.tsv
clips_path = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\clips"
validated_tsv = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\validated.tsv"

# Leer el archivo validated.tsv
df = pd.read_csv(validated_tsv, sep='\t')

# Filtrar solo los archivos que están validados
validated_files = df['path'].tolist()  # Archivos validados desde el TSV

# Procesar solo los archivos validados que están en la carpeta 'clips'
files = [file for file in os.listdir(clips_path) if file in validated_files][:20]  # Los primeros 20 archivos validados

# Procesar cada archivo
for file in files:
    audio_path = os.path.join(clips_path, file)
    
    # Obtener la transcripción del ground truth (GT) desde validated.tsv
    ground_truth = df[df['path'] == file]['sentence'].values
    if len(ground_truth) > 0:
        ground_truth = ground_truth[0]
    else:
        ground_truth = "No GT Found"  # Esto ya no debería ocurrir

    # Cargar el audio
    speech, _ = librosa.load(audio_path, sr=16000)

    # Preprocesar y transcribir usando el modelo
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    # Imprimir resultados
    print(f"Archivo: {file}")
    print(f"GT: {ground_truth}")
    print(f"Predicción: {transcription}")
    print("-" * 50)


Archivo: common_voice_ca_40865404.mp3
GT: Qui era l'anterior alcalde de Badalona?
Predicción: qui era l'anterior a alcalde de badalhoma
--------------------------------------------------
Archivo: common_voice_ca_40865427.mp3
GT: Què els va respondre l'oracle?
Predicción: què els va respondre a l'oracle
--------------------------------------------------
Archivo: common_voice_ca_40865428.mp3
GT: Com actuen les vaques?
Predicción: com ha actuan les vaques
--------------------------------------------------
Archivo: common_voice_ca_40865432.mp3
GT: Què integrava el seu llinatge?
Predicción: tente gravar el seu llinatge
--------------------------------------------------
Archivo: common_voice_ca_40865461.mp3
GT: I què va fer per assassinar-lo?
Predicción: i què va fer per assassinar-lo
--------------------------------------------------
Archivo: common_voice_ca_40865466.mp3
GT: Contra la declaració independentista, el govern d'Espanya activa la maquinària de l'estat.
Predicción: contra la decl

In [2]:
import os
import pandas as pd
import librosa
import torch
from jiwer import wer

# Ruta del directorio de clips y el archivo validated.tsv
clips_path = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\clips"
validated_tsv = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\validated.tsv"

# Leer el archivo validated.tsv
df = pd.read_csv(validated_tsv, sep='\t')

# Filtrar solo los archivos que están validados
validated_files = df['path'].tolist()  # Archivos validados desde el TSV

# Procesar solo los archivos validados que están en la carpeta 'clips'
files = [file for file in os.listdir(clips_path) if file in validated_files][:20]  # Los primeros 20 archivos validados

# Listas para almacenar las transcripciones y el ground truth
gt_transcriptions = []
model_transcriptions = []

# Procesar cada archivo
for file in files:
    audio_path = os.path.join(clips_path, file)
    
    # Obtener la transcripción del ground truth (GT) desde validated.tsv
    ground_truth = df[df['path'] == file]['sentence'].values
    if len(ground_truth) > 0:
        ground_truth = ground_truth[0]
    else:
        ground_truth = "No GT Found"  # Esto ya no debería ocurrir

    # Cargar el audio
    speech, _ = librosa.load(audio_path, sr=16000)

    # Preprocesar y transcribir usando el modelo
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    # Guardar las transcripciones para cálculo del WER
    gt_transcriptions.append(ground_truth)
    model_transcriptions.append(transcription)

    # Imprimir resultados
    print(f"Archivo: {file}")
    print(f"GT: {ground_truth}")
    print(f"Predicción: {transcription}")
    print("-" * 50)

# Calcular el WER
wer_score = wer(gt_transcriptions, model_transcriptions)
print(f"Word Error Rate (WER): {wer_score:.2f}")


Archivo: common_voice_ca_40865404.mp3
GT: Qui era l'anterior alcalde de Badalona?
Predicción: qui era l'anterior a alcalde de badalhoma
--------------------------------------------------
Archivo: common_voice_ca_40865427.mp3
GT: Què els va respondre l'oracle?
Predicción: què els va respondre a l'oracle
--------------------------------------------------
Archivo: common_voice_ca_40865428.mp3
GT: Com actuen les vaques?
Predicción: com ha actuan les vaques
--------------------------------------------------
Archivo: common_voice_ca_40865432.mp3
GT: Què integrava el seu llinatge?
Predicción: tente gravar el seu llinatge
--------------------------------------------------
Archivo: common_voice_ca_40865461.mp3
GT: I què va fer per assassinar-lo?
Predicción: i què va fer per assassinar-lo
--------------------------------------------------
Archivo: common_voice_ca_40865466.mp3
GT: Contra la declaració independentista, el govern d'Espanya activa la maquinària de l'estat.
Predicción: contra la decl

In [4]:
import os
import pandas as pd
import librosa
import torch
from jiwer import wer

# Ruta del directorio de clips y el archivo validated.tsv
clips_path = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\clips"
validated_tsv = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\validated.tsv"

# Leer el archivo validated.tsv
df = pd.read_csv(validated_tsv, sep='\t')

# Filtrar solo los archivos que están validados
validated_files = df['path'].tolist()  # Archivos validados desde el TSV

# Procesar solo los archivos validados que están en la carpeta 'clips'
files = [file for file in os.listdir(clips_path) if file in validated_files][:20]  # Los primeros 20 archivos validados

# Función para normalizar texto
def normalize_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = ''.join(c for c in text if c.isalnum() or c.isspace())  # Eliminar puntuación
    text = ' '.join(text.split())  # Eliminar espacios extras
    return text

# Listas para almacenar las transcripciones y el ground truth
gt_transcriptions = []
model_transcriptions = []

# Procesar cada archivo
for file in files:
    audio_path = os.path.join(clips_path, file)
    
    # Obtener la transcripción del ground truth (GT) desde validated.tsv
    ground_truth = df[df['path'] == file]['sentence'].values
    if len(ground_truth) > 0:
        ground_truth = ground_truth[0]
    else:
        ground_truth = "No GT Found"  # Esto ya no debería ocurrir

    # Normalizar el GT
    normalized_gt = normalize_text(ground_truth)

    # Cargar el audio
    speech, _ = librosa.load(audio_path, sr=16000)

    # Preprocesar y transcribir usando el modelo
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    # Normalizar la transcripción del modelo
    normalized_transcription = normalize_text(transcription)

    # Guardar las transcripciones normalizadas para cálculo del WER
    gt_transcriptions.append(normalized_gt)
    model_transcriptions.append(normalized_transcription)

    # Imprimir resultados
    print(f"Archivo: {file}")
    print(f"GT: {ground_truth}")
    print(f"Predicción: {transcription}")
    print("-" * 50)

# Calcular el WER con las transcripciones normalizadas
wer_score = wer(gt_transcriptions, model_transcriptions)
print(f"Word Error Rate (WER) Normalizado: {wer_score:.2f}")


Archivo: common_voice_ca_40865404.mp3
GT: Qui era l'anterior alcalde de Badalona?
Predicción: qui era l'anterior a alcalde de badalhoma
--------------------------------------------------
Archivo: common_voice_ca_40865427.mp3
GT: Què els va respondre l'oracle?
Predicción: què els va respondre a l'oracle
--------------------------------------------------
Archivo: common_voice_ca_40865428.mp3
GT: Com actuen les vaques?
Predicción: com ha actuan les vaques
--------------------------------------------------
Archivo: common_voice_ca_40865432.mp3
GT: Què integrava el seu llinatge?
Predicción: tente gravar el seu llinatge
--------------------------------------------------
Archivo: common_voice_ca_40865461.mp3
GT: I què va fer per assassinar-lo?
Predicción: i què va fer per assassinar-lo
--------------------------------------------------
Archivo: common_voice_ca_40865466.mp3
GT: Contra la declaració independentista, el govern d'Espanya activa la maquinària de l'estat.
Predicción: contra la decl

# GENDER RECOGNITION

In [20]:
import os
import pandas as pd
import librosa
import torch
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

# Paths to local model and feature extractor
local_model_dir = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\modelos\gender-recog"

from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2ForSequenceClassification

local_model_dir = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\modelos\gender-recog"

# Load the feature extractor
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(local_model_dir)

# Load the model with mismatched sizes ignored
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    local_model_dir,
    ignore_mismatched_sizes=True
)

# Set the model to evaluation mode
model.eval()


# Path to the directory of clips and validated.tsv
audio_dir = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\clips"
validated_tsv = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\validated.tsv"

# Read the validated.tsv file
df = pd.read_csv(validated_tsv, sep="\t")

# List of labels in the model
labels = ["female", "male"]

# Dictionary to map dataset gender labels to model labels
gender_mapping = {
    "male_masculine": "male",
    "female_feminine": "female",
    "other": "Unknown",
    "": "Unknown",
    "Unknown": "Unknown"
}

# Filter for files with known gender (exclude "Unknown")
valid_files = df[df['gender'].apply(lambda g: gender_mapping.get(g, "Unknown") != "Unknown")]

# Process only valid files
file_list = valid_files['path'].tolist()[:20]  # Limit to the first 20 valid files

# Evaluate each file
for file_name in file_list:
    audio_path = os.path.join(audio_dir, file_name)
    
    # Find the ground truth gender in the validated.tsv file
    gt_gender = df[df['path'] == file_name]['gender'].values
    if len(gt_gender) > 0:
        gt_gender = gender_mapping.get(gt_gender[0], "Unknown")
    else:
        continue  # Skip if there's no gender information

    # Load the audio
    speech, sr = librosa.load(audio_path, sr=16000)

    # Preprocess and predict
    inputs = feature_extractor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class = torch.argmax(logits, dim=-1).item()
    predicted_label = labels[predicted_class]

    # Check if the prediction matches the ground truth
    is_correct = (predicted_label == gt_gender)

    print(f"File: {file_name}")
    print(f"Ground Truth Gender (GT): {gt_gender}")
    print(f"Predicted Gender: {predicted_label}")
    print(f"Prediction Correct: {is_correct}")
    print("-" * 50)


File: common_voice_ca_41026074.mp3
Ground Truth Gender (GT): female
Predicted Gender: female
Prediction Correct: True
--------------------------------------------------
File: common_voice_ca_40987735.mp3
Ground Truth Gender (GT): male
Predicted Gender: male
Prediction Correct: True
--------------------------------------------------
File: common_voice_ca_40979633.mp3
Ground Truth Gender (GT): female
Predicted Gender: female
Prediction Correct: True
--------------------------------------------------
File: common_voice_ca_41041862.mp3
Ground Truth Gender (GT): female
Predicted Gender: female
Prediction Correct: True
--------------------------------------------------
File: common_voice_ca_41041883.mp3
Ground Truth Gender (GT): female
Predicted Gender: female
Prediction Correct: True
--------------------------------------------------
File: common_voice_ca_41075623.mp3
Ground Truth Gender (GT): male
Predicted Gender: male
Prediction Correct: True
---------------------------------------------

# MY OWN MODEL

In [12]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor


processor = Wav2Vec2Processor.from_pretrained(r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\modelos\wav2vec2-finetuned-ca-p3-1")
model = Wav2Vec2ForCTC.from_pretrained(r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\modelos\wav2vec2-finetuned-ca-p3-1\checkpoint-24700")



In [14]:
import os
import pandas as pd
import librosa
import torch
from jiwer import wer

# Ruta del directorio de clips y el archivo validated.tsv
clips_path = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\clips"
validated_tsv = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\validated.tsv"

# Leer el archivo validated.tsv
df = pd.read_csv(validated_tsv, sep='\t')

# Filtrar solo los archivos que están validados
validated_files = df['path'].tolist()  # Archivos validados desde el TSV

# Procesar solo los archivos validados que están en la carpeta 'clips'
files = [file for file in os.listdir(clips_path) if file in validated_files][:70]  # Los primeros 20 archivos validados

# Función para normalizar texto
def normalize_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = ''.join(c for c in text if c.isalnum() or c.isspace())  # Eliminar puntuación
    text = ' '.join(text.split())  # Eliminar espacios extras
    return text

# Listas para almacenar las transcripciones y el ground truth
gt_transcriptions = []
model_transcriptions = []

# Procesar cada archivo
for file in files:
    audio_path = os.path.join(clips_path, file)
    
    # Obtener la transcripción del ground truth (GT) desde validated.tsv
    ground_truth = df[df['path'] == file]['sentence'].values
    if len(ground_truth) > 0:
        ground_truth = ground_truth[0]
    else:
        ground_truth = "No GT Found"  # Esto ya no debería ocurrir

    # Normalizar el GT
    normalized_gt = normalize_text(ground_truth)

    # Cargar el audio
    speech, _ = librosa.load(audio_path, sr=16000)

    # Preprocesar y transcribir usando el modelo
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    # Normalizar la transcripción del modelo
    normalized_transcription = normalize_text(transcription)

    # Guardar las transcripciones normalizadas para cálculo del WER
    gt_transcriptions.append(normalized_gt)
    model_transcriptions.append(normalized_transcription)

    # Imprimir resultados
    print(f"Archivo: {file}")
    print(f"GT: {ground_truth}")
    print(f"Predicción: {transcription}")
    print("-" * 50)

# Calcular el WER con las transcripciones normalizadas
wer_score = wer(gt_transcriptions, model_transcriptions)
print(f"Word Error Rate (WER) Normalizado: {wer_score:.2f}")


Archivo: common_voice_ca_40865404.mp3
GT: Qui era l'anterior alcalde de Badalona?
Predicción: [UNK]u[UNK][UNK]ra[UNK]l'a[UNK]en[UNK]terioral[UNK]cal[UNK]de[UNK][UNK]abadaloma[UNK]
--------------------------------------------------
Archivo: common_voice_ca_40865427.mp3
GT: Què els va respondre l'oracle?
Predicción: [UNK]els[UNK]va[UNK]despon[UNK]breloracla[UNK]
--------------------------------------------------
Archivo: common_voice_ca_40865428.mp3
GT: Com actuen les vaques?
Predicción: [UNK]omvem[UNK]puent[UNK]l'r[UNK]albaces[UNK]
--------------------------------------------------
Archivo: common_voice_ca_40865432.mp3
GT: Què integrava el seu llinatge?
Predicción: [UNK]u[UNK]n[UNK]tegrelal[UNK]seu[UNK][UNK]linatga[UNK]
--------------------------------------------------
Archivo: common_voice_ca_40865461.mp3
GT: I què va fer per assassinar-lo?
Predicción: [UNK][UNK]qu[UNK][UNK]va[UNK]fer[UNK]per[UNK]ra[UNK][UNK]escina-lo[UNK]
--------------------------------------------------
Archivo: co

# ESP TO CA

In [1]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor


processor = Wav2Vec2Processor.from_pretrained(r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\FINETUNE-ESP-CA\modelesp")
model = Wav2Vec2ForCTC.from_pretrained(r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\FINETUNE-ESP-CA\modelesp")



d:\Users\Usuario\anaconda3\envs\new_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import pandas as pd
import librosa
import torch
from jiwer import wer

# Ruta del directorio de clips y el archivo validated.tsv
clips_path = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\clips"
validated_tsv = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\validated.tsv"

# Leer el archivo validated.tsv
df = pd.read_csv(validated_tsv, sep='\t')

# Filtrar solo los archivos que están validados
validated_files = df['path'].tolist()  # Archivos validados desde el TSV

# Procesar solo los archivos validados que están en la carpeta 'clips'
files = [file for file in os.listdir(clips_path) if file in validated_files][:70]  # Los primeros 20 archivos validados

# Función para normalizar texto
def normalize_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = ''.join(c for c in text if c.isalnum() or c.isspace())  # Eliminar puntuación
    text = ' '.join(text.split())  # Eliminar espacios extras
    return text

# Listas para almacenar las transcripciones y el ground truth
gt_transcriptions = []
model_transcriptions = []

# Procesar cada archivo
for file in files:
    audio_path = os.path.join(clips_path, file)
    
    # Obtener la transcripción del ground truth (GT) desde validated.tsv
    ground_truth = df[df['path'] == file]['sentence'].values
    if len(ground_truth) > 0:
        ground_truth = ground_truth[0]
    else:
        ground_truth = "No GT Found"  # Esto ya no debería ocurrir

    # Normalizar el GT
    normalized_gt = normalize_text(ground_truth)

    # Cargar el audio
    speech, _ = librosa.load(audio_path, sr=16000)

    # Preprocesar y transcribir usando el modelo
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    # Normalizar la transcripción del modelo
    normalized_transcription = normalize_text(transcription)

    # Guardar las transcripciones normalizadas para cálculo del WER
    gt_transcriptions.append(normalized_gt)
    model_transcriptions.append(normalized_transcription)

    # Imprimir resultados
    print(f"Archivo: {file}")
    print(f"GT: {ground_truth}")
    print(f"Predicción: {transcription}")
    print("-" * 50)

# Calcular el WER con las transcripciones normalizadas
wer_score = wer(gt_transcriptions, model_transcriptions)
print(f"Word Error Rate (WER) Normalizado: {wer_score:.2f}")


Archivo: common_voice_ca_40865404.mp3
GT: Qui era l'anterior alcalde de Badalona?
Predicción: quiere el anterior alcalda de badalona
--------------------------------------------------
Archivo: common_voice_ca_40865427.mp3
GT: Què els va respondre l'oracle?
Predicción: que els barraspondr al uracla
--------------------------------------------------
Archivo: common_voice_ca_40865428.mp3
GT: Com actuen les vaques?
Predicción: cómo hactú en las vacas
--------------------------------------------------
Archivo: common_voice_ca_40865432.mp3
GT: Què integrava el seu llinatge?
Predicción: teintagrabaal soulinaya
--------------------------------------------------
Archivo: common_voice_ca_40865461.mp3
GT: I què va fer per assassinar-lo?
Predicción: y qué bafe para asasinarlo
--------------------------------------------------
Archivo: common_voice_ca_40865466.mp3
GT: Contra la declaració independentista, el govern d'Espanya activa la maquinària de l'estat.
Predicción: contra la declaración de band

# ENG TO CAT

In [1]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor


processor = Wav2Vec2Processor.from_pretrained(r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\FINETUNE-ENG-CA\model")
model = Wav2Vec2ForCTC.from_pretrained(r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\FINETUNE-ENG-CA\model")



d:\Users\Usuario\anaconda3\envs\new_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\FINETUNE-ENG-CA\model and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import os
import pandas as pd
import librosa
import torch
from jiwer import wer

# Ruta del directorio de clips y el archivo validated.tsv
clips_path = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\clips"
validated_tsv = r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca\validated.tsv"

# Leer el archivo validated.tsv
df = pd.read_csv(validated_tsv, sep='\t')

# Filtrar solo los archivos que están validados
validated_files = df['path'].tolist()  # Archivos validados desde el TSV

# Procesar solo los archivos validados que están en la carpeta 'clips'
files = [file for file in os.listdir(clips_path) if file in validated_files][:70]  # Los primeros 20 archivos validados

# Función para normalizar texto
def normalize_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = ''.join(c for c in text if c.isalnum() or c.isspace())  # Eliminar puntuación
    text = ' '.join(text.split())  # Eliminar espacios extras
    return text

# Listas para almacenar las transcripciones y el ground truth
gt_transcriptions = []
model_transcriptions = []

# Procesar cada archivo
for file in files:
    audio_path = os.path.join(clips_path, file)
    
    # Obtener la transcripción del ground truth (GT) desde validated.tsv
    ground_truth = df[df['path'] == file]['sentence'].values
    if len(ground_truth) > 0:
        ground_truth = ground_truth[0]
    else:
        ground_truth = "No GT Found"  # Esto ya no debería ocurrir

    # Normalizar el GT
    normalized_gt = normalize_text(ground_truth)

    # Cargar el audio
    speech, _ = librosa.load(audio_path, sr=16000)

    # Preprocesar y transcribir usando el modelo
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    # Normalizar la transcripción del modelo
    normalized_transcription = normalize_text(transcription)

    # Guardar las transcripciones normalizadas para cálculo del WER
    gt_transcriptions.append(normalized_gt)
    model_transcriptions.append(normalized_transcription)

    # Imprimir resultados
    print(f"Archivo: {file}")
    print(f"GT: {ground_truth}")
    print(f"Predicción: {transcription}")
    print("-" * 50)

# Calcular el WER con las transcripciones normalizadas
wer_score = wer(gt_transcriptions, model_transcriptions)
print(f"Word Error Rate (WER) Normalizado: {wer_score:.2f}")


Archivo: common_voice_ca_40865404.mp3
GT: Qui era l'anterior alcalde de Badalona?
Predicción: GERANT YOUR ANCALDEMAMADELOME
--------------------------------------------------
Archivo: common_voice_ca_40865427.mp3
GT: Què els va respondre l'oracle?
Predicción: GETS MA AS PORM THAT I RACKLER
--------------------------------------------------
Archivo: common_voice_ca_40865428.mp3
GT: Com actuen les vaques?
Predicción: COBAC TO I LI LAKAS
--------------------------------------------------
Archivo: common_voice_ca_40865432.mp3
GT: Què integrava el seu llinatge?
Predicción: DEN TARA ALSO DE NADEA
--------------------------------------------------
Archivo: common_voice_ca_40865461.mp3
GT: I què va fer per assassinar-lo?
Predicción: HE GAVE UP A BARASAS IN A RUM
--------------------------------------------------
Archivo: common_voice_ca_40865466.mp3
GT: Contra la declaració independentista, el govern d'Espanya activa la maquinària de l'estat.
Predicción: CONTELA RACARASINDABUNDANTHISTA I WIL R

# ACCENT RECOGNITION

In [4]:
import os
import pandas as pd
from datasets import Dataset, Audio, concatenate_datasets
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification
import torch
import numpy as np
from sklearn.metrics import classification_report

# List of dataset directories to process
data_dirs = [
    r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca8\ca",
    r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\ca5\ca"  # Add more directories as needed
]

# Function to load and process a batch
def load_and_process_batch(data_dir, accent_label_map):
    # Load metadata
    metadata_path = os.path.join(data_dir, "validated.tsv")
    metadata = pd.read_csv(metadata_path, sep="\t")

    # Filter for necessary columns
    metadata = metadata[["path", "sentence", "accents"]]

    # Filter for target accents
    metadata = metadata[metadata["accents"].isin(accent_label_map.keys())]

    # Map accents to labels
    metadata["label"] = metadata["accents"].map(accent_label_map)

    # Update path column to point to the audio files
    metadata["path"] = metadata["path"].apply(lambda x: os.path.join(data_dir, "clips", x))

    # Convert to Hugging Face Dataset
    dataset = Dataset.from_pandas(metadata)

    # Cast "path" column to audio
    dataset = dataset.cast_column("path", Audio(sampling_rate=16_000))

    return dataset

# Load the label map from the training process
accent_label_map = {
    "central": 0,
    "valencià,La Vall d'Albaida": 1,
    "nord-occidental,Tortosí": 2,
    "balear": 3,
    "septentrional": 4
}

# Process datasets from all directories
datasets = [load_and_process_batch(data_dir, accent_label_map) for data_dir in data_dirs]

# Concatenate all datasets
combined_dataset = concatenate_datasets(datasets)

print("Combined dataset prepared:", combined_dataset)


Combined dataset prepared: Dataset({
    features: ['path', 'sentence', 'accents', 'label', '__index_level_0__'],
    num_rows: 7851
})


In [7]:
# Load trained model and processor

model = Wav2Vec2ForSequenceClassification.from_pretrained(r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\modelos\wav2vec2-accent-classification\checkpoint-7100")
processor = Wav2Vec2Processor.from_pretrained(r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\modelos\wav2vec2-accent-classification")

# Set model to evaluation mode
model.eval()

# Function for inference
def predict(dataset):
    predictions = []
    ground_truths = []
    for item in dataset:
        # Process audio and ground truth label
        input_audio = item["path"]["array"]
        ground_truths.append(item["label"])

        # Preprocess input for the model
        inputs = processor(
            input_audio,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True
        )

        # Perform inference
        with torch.no_grad():
            logits = model(**inputs).logits

        # Get predicted label
        predicted_label = torch.argmax(logits, dim=-1).item()
        predictions.append(predicted_label)
    
    return predictions, ground_truths

# Run inference on the new dataset
predictions, ground_truths = predict(combined_dataset)


In [ ]:
# Load trained model and processor
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\modelos\wav2vec2-accent-classification\checkpoint-7100"
)
processor = Wav2Vec2Processor.from_pretrained(
    r"D:\Usuario\Desktop\Uni\4th year\Advanced Machine Learning\PROJECT GROUP\modelos\wav2vec2-accent-classification"
)

# Set model to evaluation mode
model.eval()

# Function for inference on the first N samples
def predict_first_n_samples(dataset, n=10):
    predictions = []
    ground_truths = []
    for idx, item in enumerate(dataset.select(range(n))):
        # Process audio and ground truth label
        input_audio = item["path"]["array"]
        ground_truth = item["label"]

        # Preprocess input for the model
        inputs = processor(
            input_audio,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True
        )

        # Perform inference
        with torch.no_grad():
            logits = model(**inputs).logits

        # Get predicted label
        predicted_label = torch.argmax(logits, dim=-1).item()
        predictions.append(predicted_label)
        ground_truths.append(ground_truth)

        # Print the ground truth and prediction
        print(f"Sample {idx + 1}: Ground Truth: {ground_truth}, Prediction: {predicted_label}")
    
    return predictions, ground_truths

# Run inference on the first 100 samples of the new dataset
predictions, ground_truths = predict_first_n_samples(combined_dataset, n=10)


Sample 2: Ground Truth: 0, Prediction: 0
Sample 3: Ground Truth: 0, Prediction: 0
Sample 4: Ground Truth: 0, Prediction: 0
Sample 5: Ground Truth: 0, Prediction: 0
Sample 6: Ground Truth: 3, Prediction: 3
